In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from pathlib import Path
import csv
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import RMSprop
from collections import Counter
from itertools import repeat, chain

In [3]:
hp_df = pd.read_csv(Path('./hp_df.csv'))
hp_df.drop(columns='Unnamed: 0',inplace=True)
hp_df.head()

,word
0,THE
1,BOY
2,WHO
3,LIVED
4,Mr.


In [20]:
words=hp_df['word'].to_list()
words[0:5]

['THE', 'BOY', 'WHO', 'LIVED', 'Mr.']

In [57]:
word_sort = sorted(words, key = words.count, reverse = True)
word_sort[0:10]

['the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']

In [47]:
word_sort.index("summer...”")

83162

In [76]:
word_sort[3651]

'the'

In [103]:
weighted_words = []
weighted_count = []
count = 0
for i in word_sort:
    if count == 0:
        weighted_words.append(i)
        count+=1
    elif word_sort.index(i) == 83162:
        #for j in weighted_count:
            #if j == count:
                #count = j + 0.00000000001 * word_sort.index(i)
                #break
        weighted_count.append(count)
        count = 1
        weighted_words.append(i)
        weighted_count.append(count)
    elif i != weighted_words[-1]:
        #for j in weighted_count:
            #if j == count:
                #count = j + 0.00000000001 * word_sort.index(i)
                #break
        weighted_count.append(count)
        count = 1
        weighted_words.append(i)
    else:
        count+=1

display(weighted_words[:10])
display(weighted_count[:10])
display(weighted_words[-10:])
display(weighted_count[-10:])

['the', 'and', 'to', 'a', 'Harry', 'of', 'was', 'he', 'in', 'his']

[3652, 2137, 1827, 1578, 1252, 1233, 1150, 1020, 898, 892]

['mustached,',
 'manner',
 'speaking,”',
 'day.”',
 'summer,',
 'holiday,”',
 'uncertainly',
 'unpleasant.',
 'spreading',
 'summer...”']

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [113]:
hp_df = pd.DataFrame(columns=['word','id'])
hp_df['word'] = weighted_words
hp_df['id'] = weighted_count
#hp_df.loc[hp_df['word'] == 'Potter']
hp_df.head(50)

,word,id
0,the,3652
1,and,2137
2,to,1827
3,a,1578
4,Harry,1252
5,of,1233
6,was,1150
7,he,1020
8,in,898
9,his,892


In [116]:
hp_df[hp_df.duplicated(subset='word')]

,word,id
25,Potter,1
26,Stone,1
27,Potter,1
28,Stone,1
29,Potter,3
...,...,...
35663,"sweater,",1
35664,"nothing,",1
35665,family!”,1
35666,"will,”",1


In [125]:
hp_df = hp_df.groupby('word').sum()
hp_df = hp_df.sort_values(by=['id'],ascending=False)
hp_df.head(50)

,id
word,
the,3652
and,2137
to,1827
a,1578
Harry,1252
of,1233
was,1150
he,1020
in,898


In [126]:
hp_df.to_csv('./hp_df_unique.csv')

In [73]:
LENGTH_WORD = 5
next_words = []
prev_words = []
for j in range(len(words) - LENGTH_WORD):
     prev_words.append(words[j:j + LENGTH_WORD])
     next_words.append(words[j + LENGTH_WORD])
print(prev_words[0])
print(next_words[0])

['THE', 'BOY', 'WHO', 'LIVED', 'Mr.']
and


In [75]:
X = np.zeros((len(prev_words), LENGTH_WORD, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
   for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
   Y[i, unique_word_index[next_words[i]]] = 1

In [81]:
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH_WORD, len(unique_words))))
model.add(Dense(len(unique_words),activation='softmax'))

In [84]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

/Users/robbyodum/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
618/618 [==============================] - 338s 541ms/step - loss: 6.8397 - accuracy: 0.1062 - val_loss: 6.6642 - val_accuracy: 0.1222
Epoch 2/2
618/618 [==============================] - 294s 477ms/step - loss: 6.3905 - accuracy: 0.1554 - val_loss: 7.0004 - val_accuracy: 0.1306


In [85]:
model.predict(X)

2599/2599 [==============================] - 244s 94ms/step


array([[3.3681274e-06, 1.6582164e-06, 6.7449719e-07, ..., 3.7319160e-06,
        3.6585791e-06, 7.9419340e-07],
       [2.2907639e-06, 1.5850809e-06, 1.2439924e-06, ..., 1.6425649e-06,
        3.7900300e-06, 9.9285137e-07],
       [3.0714400e-06, 1.1796001e-06, 7.9796240e-07, ..., 2.0622442e-06,
        2.6972048e-06, 9.2389314e-07],
       ...,
       [2.7371241e-06, 1.1516806e-06, 1.2824618e-06, ..., 2.4652106e-06,
        2.5429724e-06, 9.1476926e-07],
       [2.0623968e-06, 1.6584582e-06, 1.5313720e-06, ..., 4.5622287e-06,
        3.1612574e-06, 1.0157548e-06],
       [3.7905697e-06, 7.4769554e-07, 1.0440364e-06, ..., 1.7161175e-06,
        2.3543050e-06, 7.6218947e-07]], dtype=float32)

In [36]:
words_ex = words[0:5]
display(words_ex)
num_words = []
for char in words_ex:
    word = []
    for i in char:
        word.append(ord(i))
    num_words.append(word)
display(num_words)

['THE', 'BOY', 'WHO', 'LIVED', 'Mr.']

[[84, 72, 69], [66, 79, 89], [87, 72, 79], [76, 73, 86, 69, 68], [77, 114, 46]]

In [57]:
f.close()

In [56]:
# Puts into dict format
hp_dict = {'word':words}

In [86]:
# Creates df of word numbers
hp_df = pd.DataFrame(hp_dict,columns=['word'])
hp_df.head()

,word
0,THE
1,BOY
2,WHO
3,LIVED
4,Mr.


In [87]:
hp_df.to_csv('hp_df.csv')

In [69]:
hp_df = pd.get_dummies(hp_df.word)
hp_df.head()

,!,!”,"""","""...",(Bewitch,(Grade,(Neville,(Order,(Slytherin,(as,...,“what’s,“where,“you,"“you,",“you’ll,“—,”,”).,•k,■”
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
hp_df.columns

Index(['!', '!”', '"', '"...', '(Bewitch', '(Grade', '(Neville', '(Order',
       '(Slytherin', '(as',
       ...
       '“what’s', '“where', '“you', '“you,', '“you’ll', '“—', '”', '”).', '•k',
       '■”'],
      dtype='object', length=11980)

In [46]:
type(hp_df['word'][0][0])

int

In [71]:
# Adds predictor word column
hp_df['new_word'] = hp_df['word'].shift(1)
hp_df.head()

,!,!”,"""","""...",(Bewitch,(Grade,(Neville,(Order,(Slytherin,(as,...,“where,“you,"“you,",“you’ll,“—,”,”).,•k,■”,new_word
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [48]:
X,y = hp_df['word'], hp_df['new_word']

In [49]:
split = int(len(X)*0.7)
# Splits train and test data
X_train=X[:split]
X_test=X[split:]
y_train=y[:split]
y_test=y[split:]
display(X_train.head())
display(X_test.head())
display(y_train.head())
display(y_test.head())

0            [84, 72, 69]
1            [66, 79, 89]
2            [87, 72, 79]
3    [76, 73, 86, 69, 68]
4           [77, 114, 46]
Name: word, dtype: object

58214          [84, 104, 101]
58215    [114, 101, 115, 116]
58216              [111, 102]
58217         [116, 104, 101]
58218     [116, 101, 97, 109]
Name: word, dtype: object

0                     NaN
1            [84, 72, 69]
2            [66, 79, 89]
3            [87, 72, 79]
4    [76, 73, 86, 69, 68]
Name: new_word, dtype: object

58214            [46, 46, 46]
58215          [84, 104, 101]
58216    [114, 101, 115, 116]
58217              [111, 102]
58218         [116, 104, 101]
Name: new_word, dtype: object

In [50]:
# Creates neural network
model=Sequential()
# Defines and adds LSTM layer
first_lstm_layer=LSTM(units=30, return_sequences=True)
model.add(first_lstm_layer)
# Randomly drops 20% of defined nodes
first_dropout_layer=Dropout(0.2)
# Defines and adds LSTM layer
model.add(first_dropout_layer)
second_lstm_layer=LSTM(units=30, return_sequences=True)
model.add(second_lstm_layer)
# Randomly drops 20% of defined nodes
second_dropout_layer=Dropout(0.2)
# Defines and adds LSTM layer
model.add(second_dropout_layer)
third_lstm_layer=LSTM(units=30)
model.add(third_lstm_layer)
# Defines and adds output layer with linear activation
output_layer=Dense(units=1, activation='linear')
model.add(output_layer)

In [51]:
# Compiles model
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_squared_error')

In [52]:
# Provides summary of model
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [53]:
model.fit(X_train, y_train, epochs=50)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).